In [222]:
import httpx
import time
import json
import os

In [223]:
tid = 35018847

In [224]:
res = httpx.get(f'https://bbs.nga.cn/read.php?tid={tid}')
first_request_time: int = int(time.time())

In [225]:
header_parse = res.headers['set-cookie'].split(';')
header_parse = [i.strip() for i in header_parse]

In [226]:
header_parse

['lastvisit=1673599970',
 'path=/',
 'domain=bbs.nga.cn',
 'secure, lastpath=/read.php?tid=35018847',
 'path=/',
 'domain=bbs.nga.cn',
 'secure, ngaPassportUid=guest063c11be2c45a3',
 'expires=Fri, 13-Jan-2023 18:52:50 GMT',
 'Max-Age=36000',
 'path=/',
 'domain=.nga.cn']

In [216]:
cookies = {"guestJs": str(first_request_time)}
for header in header_parse:
    if header.startswith('lastvisit'):
        k, v, *_ = header.split('=')
        cookies['lastvisit'] = v
    elif header.startswith("secure"):
        k, v, *_ = header.split('=')
        cookies['ngaPassportUid'] = v

In [217]:
res2 = httpx.get(f'https://bbs.nga.cn/read.php?tid={tid}&lite=js', cookies=cookies)

In [221]:
res2.text

'window.script_muti_get_var_store={"data":{"__GLOBAL":{"_ATTACH_BASE_VIEW":"img.nga.178.com/attachments"},"__U":{"41377355":{"uid":41377355,"username":"UID:41377355","credit":169213952,"medal":"","reputation":"61_3,75_30,-7_105,91_15,246_15,134_15","groupid":-1,"memberid":39,"avatar":"","yz":4,"site":"","honor":"","regdate":1494985570,"mute_time":"","postnum":2087,"rvrc":10,"money":31100,"thisvisit":1673545220,"signature":"","nickname":"","bit_data":169213952},"60969726":{"uid":60969726,"username":"UID:60969726","credit":169082880,"medal":"","reputation":"61_13,22_50,450012_30,450014_20,74_15,-7_60","groupid":-1,"memberid":39,"avatar":"","yz":4,"site":"","honor":"","regdate":1568573241,"mute_time":"","postnum":208,"rvrc":10,"money":7125,"thisvisit":1673568441,"signature":"","nickname":"","bit_data":169082880},"2365557":{"uid":2365557,"username":"UID:2365557","credit":169082880,"medal":"","reputation":"61_22,19_30,131_30","groupid":-1,"memberid":39,"avatar":"","yz":4,"site":"","honor":"

In [218]:
print(len(res2.content))

15967


In [219]:
data = json.loads(res2.text[len("window.script_muti_get_var_store="):])['data']

In [220]:
img_base = data['__GLOBAL']['_ATTACH_BASE_VIEW']
reply = data["__R"]
post = data['__T']

In [157]:
reply_contains_img = []
for _, d in reply.items():
    if d.get('attachs') is not None:
        reply_contains_img.append(d)

In [192]:
base_dir = post['subject']
os.makedirs(f'content/{base_dir}', exist_ok=True)
for each in reply_contains_img:
    for _, each_content in each['attachs'].items():
        sub = each_content['attachurl']
        url = f'https://{img_base}/{sub}'
        r = httpx.get(url)
        img_name = sub.split('/')[-1]
        img_path = f'content/{base_dir}/{img_name}'
        with open(img_path, 'wb') as f:
            f.write(r.content)

In [228]:
from utils import  parse_header

In [227]:
res2.headers

Headers([('date', 'Fri, 13 Jan 2023 08:48:56 GMT'), ('content-type', 'text/javascript; charset=GBK'), ('transfer-encoding', 'chunked'), ('connection', 'keep-alive'), ('server', 'nginx'), ('strict-transport-security', 'max-age=604800'), ('x-nga-server', '53'), ('set-cookie', 'lastvisit=1673599737; path=/; domain=bbs.nga.cn; secure'), ('set-cookie', 'lastpath=/read.php?tid=35018847&lite=js; path=/; domain=bbs.nga.cn; secure'), ('content-encoding', 'gzip'), ('x-via', '1.1 PS-PEK-01zM8144:8 (Cdn Cache Server V2.0)'), ('x-ws-request-id', '63c11af8_PS-PEK-01mMC145_10059-57312')])

In [ ]:
parse_header(res2.headers, 1, )